In [1]:
# to drop:
# nuclear
# oil
# mixed and renewable hydro

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco
import os

In [3]:
path = '/Users/ariana/desktop/historical_tech/raw data/irena'
os.chdir(path)

In [4]:
# https://pxweb.irena.org/pxweb/en/IRENASTAT

In [5]:
irena = os.listdir(path)
for file in irena:
    if '.xlsx' not in file:
        irena.remove(file)

In [6]:
def read_irena(file_name):
    file = os.path.abspath(file_name)
    df = pd.read_excel(file, header=2, na_values='..', skipfooter=26)
    df.dropna(how='all', inplace=True)
    df.rename(columns={'Unnamed: 0': 'Country Name'}, inplace=True)
    df['Data Source'] = 'IRENA'
    df['Spatial Scale'] = 'National'
    df['Metric'] = 'Installed electricity capacity'
    df['Unit'] = 'MW'
    iso2 = []
    for country in df['Country Name']:
        iso2.append(coco.convert(names=country, to='iso2'))
    df['Country Code'] = iso2
    tech = df['Unnamed: 1'].iloc[0]
    grid = df['Unnamed: 2'].iloc[0]
    if tech == 'Onshore wind energy' or tech == 'Offshore wind energy':
        tech_name = grid + ' ' + tech
    else:
        tech_name = tech
    df['Technology Name'] = tech_name.title()
    df.drop(columns=['Unnamed: 1', 'Unnamed: 2'], inplace=True)
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', inplace=True)
    return df

In [7]:
irena_df = []
for tech in irena:
    df = read_irena(tech)
    irena_df.append(df)

In [8]:
irena_concat = pd.concat(irena_df)

In [9]:
omit = []
for col in irena_concat.columns:
    if len(col) != 4 or col == 'Unit':
        omit.append(col)
empty_rows = irena_concat.drop(columns=omit)
empty_rows.dropna(how='all', inplace=True)
na_idx = []
for country in irena_concat.index:
    if country not in empty_rows.index:
        na_idx.append(country)
irena_concat.drop(na_idx, inplace=True)

In [12]:
irena_concat[irena_concat['Technology Name']=='On-Grid Offshore Wind Energy']

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2018,2019,2020,2021,Data Source,Spatial Scale,Metric,Unit,Country Code,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
On-Grid Offshore Wind Energy_Installed electricity capacity_BE,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1185.9,1555.500,2261.80,2261.80,IRENA,National,Installed electricity capacity,MW,BE,On-Grid Offshore Wind Energy
On-Grid Offshore Wind Energy_Installed electricity capacity_CN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.50,1.50,...,4588.0,5930.000,8990.00,26390.00,IRENA,National,Installed electricity capacity,MW,CN,On-Grid Offshore Wind Energy
On-Grid Offshore Wind Energy_Installed electricity capacity_TW,Chinese Taipei,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,128.000,128.00,237.20,IRENA,National,Installed electricity capacity,MW,TW,On-Grid Offshore Wind Energy
On-Grid Offshore Wind Energy_Installed electricity capacity_DK,Denmark,49.95,49.95,213.95,423.35,423.35,423.35,423.35,423.35,423.35,...,1700.8,1700.800,1700.80,2305.80,IRENA,National,Installed electricity capacity,MW,DK,On-Grid Offshore Wind Energy
On-Grid Offshore Wind Energy_Installed electricity capacity_FI,Finland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.00,...,72.7,73.000,73.00,73.00,IRENA,National,Installed electricity capacity,MW,FI,On-Grid Offshore Wind Energy
On-Grid Offshore Wind Energy_Installed electricity capacity_FR,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.000,2.00,2.00,IRENA,National,Installed electricity capacity,MW,FR,On-Grid Offshore Wind Energy
On-Grid Offshore Wind Energy_Installed electricity capacity_DE,Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6393.0,7555.000,7774.00,7747.00,IRENA,National,Installed electricity capacity,MW,DE,On-Grid Offshore Wind Energy
On-Grid Offshore Wind Energy_Installed electricity capacity_IE,Ireland,NaN,NaN,NaN,NaN,25.20,25.20,25.20,25.20,25.20,...,25.2,25.200,25.20,25.20,IRENA,National,Installed electricity capacity,MW,IE,On-Grid Offshore Wind Energy
On-Grid Offshore Wind Energy_Installed electricity capacity_JP,Japan,NaN,NaN,NaN,1.20,1.20,1.20,1.20,11.20,11.20,...,65.0,65.000,65.00,65.00,IRENA,National,Installed electricity capacity,MW,JP,On-Grid Offshore Wind Energy


In [11]:
irena_concat.to_csv('/Users/ariana/desktop/historical_tech/cleaned data/irena.csv')